In [1]:
%reset -f

In [2]:
!pip install /Users/dhritiman/Documents/projects/mit-main/nobrainer-dev/nobrainer-exp/

Processing /Users/dhritiman/Documents/projects/mit-main/nobrainer-dev/nobrainer-exp
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for nobrainer: filename=nobrainer-0.1a1+0.g8e89492.dirty-py3-none-any.whl size=95676 sha256=b4a12314bbb13974212ecdea90834e72fdf218e29b038723a85df2fea00f7eb6
  Stored in directory: /Users/dhritiman/Library/Caches/pip/wheels/1d/3e/e7/ae4b7c6513d2cff5bfeea6eb263919c60b24b717b55157f422
Successfully built nobrainer
  Attempting uninstall: nobrainer
    Found existing installation:

In [3]:
import nobrainer
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

in dataset file


In [5]:
print(nobrainer.__version__)

0.1a1+0.g8e89492.dirty


In [6]:
!which nobrainer

/Users/dhritiman/opt/anaconda3/envs/nobrainer-dev/bin/nobrainer


In [7]:
import random

csv_of_filepaths = nobrainer.utils.get_data()
filepaths = nobrainer.io.read_csv(csv_of_filepaths)

# Add random boolean values
filepaths = [(x, random.choice([0, 1])) for x, _ in filepaths]

train_paths = filepaths[:9]
evaluate_paths = filepaths[9:]

335872/335557 [==============================] - 0s 0us/step


In [8]:
filepaths

[('/var/folders/sh/dz5rd4yx2gs2cw1rgr6lxrvw0000gn/T/nobrainer-data/datasets/sub-01_t1.mgz',
  0),
 ('/var/folders/sh/dz5rd4yx2gs2cw1rgr6lxrvw0000gn/T/nobrainer-data/datasets/sub-02_t1.mgz',
  1),
 ('/var/folders/sh/dz5rd4yx2gs2cw1rgr6lxrvw0000gn/T/nobrainer-data/datasets/sub-03_t1.mgz',
  0),
 ('/var/folders/sh/dz5rd4yx2gs2cw1rgr6lxrvw0000gn/T/nobrainer-data/datasets/sub-04_t1.mgz',
  1),
 ('/var/folders/sh/dz5rd4yx2gs2cw1rgr6lxrvw0000gn/T/nobrainer-data/datasets/sub-05_t1.mgz',
  0),
 ('/var/folders/sh/dz5rd4yx2gs2cw1rgr6lxrvw0000gn/T/nobrainer-data/datasets/sub-06_t1.mgz',
  1),
 ('/var/folders/sh/dz5rd4yx2gs2cw1rgr6lxrvw0000gn/T/nobrainer-data/datasets/sub-07_t1.mgz',
  1),
 ('/var/folders/sh/dz5rd4yx2gs2cw1rgr6lxrvw0000gn/T/nobrainer-data/datasets/sub-08_t1.mgz',
  1),
 ('/var/folders/sh/dz5rd4yx2gs2cw1rgr6lxrvw0000gn/T/nobrainer-data/datasets/sub-09_t1.mgz',
  0),
 ('/var/folders/sh/dz5rd4yx2gs2cw1rgr6lxrvw0000gn/T/nobrainer-data/datasets/sub-10_t1.mgz',
  0)]

In [9]:
# Verify that all volumes have the same shape and that labels are integer-ish.

invalid = nobrainer.io.verify_features_labels(train_paths)
assert not invalid

invalid = nobrainer.io.verify_features_labels(evaluate_paths)
assert not invalid

Verifying 9 examples
9/9 [==============================] - 0s 2ms/step
Verifying 1 examples
1/1 [==============================] - 0s 111ms/step


In [10]:
!mkdir -p data

In [11]:
# Convert training and evaluation data to TFRecords.

nobrainer.tfrecord.write(
    features_labels=train_paths,
    filename_template='data/data-train_shard-{shard:03d}.tfrec',
    examples_per_shard=3)

nobrainer.tfrecord.write(
    features_labels=evaluate_paths,
    filename_template='data/data-evaluate_shard-{shard:03d}.tfrec',
    examples_per_shard=1)

1/1 [==============================] - 3s 3s/step


In [12]:
!ls data

data-evaluate_shard-000.tfrec data-train_shard-001.tfrec
data-train_shard-000.tfrec    data-train_shard-002.tfrec


In [13]:
n_classes = 1
batch_size = 1
volume_shape = (256, 256, 256)
block_shape = (8, 8, 8)
n_epochs = None
num_parallel_calls = 2

In [14]:
dataset_train_1 = nobrainer.dataset.get_dataset(
    file_pattern="data/data-train_shard*.tfrec",
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    scalar_label=True,
    augment=True,
    augmentType=True,
    block_shape=block_shape,
    n_epochs=n_epochs,
    num_parallel_calls=num_parallel_calls,
)

dataset_train_2 = nobrainer.dataset.get_dataset(
    file_pattern="data/data-train_shard*.tfrec",
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    scalar_label=True,
    augment=True,
    augmentType=False,
    block_shape=block_shape,
    n_epochs=n_epochs,
    num_parallel_calls=num_parallel_calls,
)

dataset_evaluate = nobrainer.dataset.get_dataset(
    file_pattern="data/data-evaluate_shard-*.tfrec",
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    scalar_label=True,
    augment=False,
    block_shape=block_shape,
    n_epochs=1,
    num_parallel_calls=num_parallel_calls,
)

after get dataset
before calling augment
scalar true
augment type: true, add Gaussian Noise
after get dataset
before calling augment
scalar true
augment type: false, add random transform
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
after get dataset
before calling augment


In [15]:
dataset_train_1

<RepeatDataset shapes: ((1, 8, 8, 8, 1), (1,)), types: (tf.float32, tf.float32)>

In [16]:
dataset_train_2

<RepeatDataset shapes: ((1, 8, 8, 8, 1), (1,)), types: (tf.float32, tf.float32)>

In [17]:
dataset_evaluate

<RepeatDataset shapes: ((1, 8, 8, 8, 1), (1,)), types: (tf.float32, tf.float32)>

In [18]:
dataset_train_1 = dataset_train_1.map(lambda x, y:x)

dataset_train_1

<MapDataset shapes: (1, 8, 8, 8, 1), types: tf.float32>

In [19]:
dataset_train_2 = dataset_train_2.map(lambda x, y:x)
dataset_train_2

<MapDataset shapes: (1, 8, 8, 8, 1), types: tf.float32>

In [20]:
from nobrainer.models.brainsiam import brainsiam
from nobrainer import training

In [21]:
projection_dim = 2048
latent_dim = 512
weight_decay = 0.0005

In [22]:
#/Users/dhritiman/Documents/projects/mit-main/nobrainer-dev/nobrainer/models/wip_simsiam_brain.py

encoder, predictor = brainsiam(n_classes=n_classes, input_shape=(*block_shape, 1),weight_decay = weight_decay,
    projection_dim = projection_dim,
    latent_dim = latent_dim,
    )

projection dimension is:  2048
latent dimension:  512
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 8, 8, 8, 1)]      0         
_________________________________________________________________
highresnet (Functional)      (None, 8, 8, 8, 1)        874785    
_________________________________________________________________
backbone_pool (GlobalAverage (None, 1)                 0         
_________________________________________________________________
dense (Dense)                (None, 2048)              2048      
_________________________________________________________________
batch_normalization_18 (Batc (None, 2048)              8192      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 2048)              0         
_________________________________________________________________
dense

In [23]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 8, 8, 8, 1)]      0         
_________________________________________________________________
highresnet (Functional)      (None, 8, 8, 8, 1)        874785    
_________________________________________________________________
backbone_pool (GlobalAverage (None, 1)                 0         
_________________________________________________________________
dense (Dense)                (None, 2048)              2048      
_________________________________________________________________
batch_normalization_18 (Batc (None, 2048)              8192      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4194

In [24]:
augment_noise_rigid = tf.data.Dataset.zip((dataset_train_1, dataset_train_2))
print(augment_noise_rigid)

<ZipDataset shapes: ((1, 8, 8, 8, 1), (1, 8, 8, 8, 1)), types: (tf.float32, tf.float32)>


In [25]:
steps = nobrainer.dataset.get_steps_per_epoch(
    n_volumes = len(train_paths),
    volume_shape=volume_shape,
    block_shape=block_shape,
    batch_size=batch_size
)

steps = 128

steps

128

In [26]:
lr_decayed_fn = tf.keras.experimental.CosineDecay(
    initial_learning_rate=0.01, decay_steps=steps #original initial rate = 0.001
)

In [27]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="loss", patience=5, restore_best_weights=True
)

In [28]:
checkpoint_path = "/gdrive/MyDrive/training_noise_rigid/cp-{epoch:04d}.ckpt"

In [29]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1)

In [30]:
EPOCHS = 5


In [31]:
model = training.BrainSiamese(encoder, predictor)  #SimSiam(encoder, predictor)

In [32]:
model.compile(optimizer=tf.keras.optimizers.SGD(lr_decayed_fn, momentum=0.6))

In [33]:
history = model.fit(augment_noise_rigid, epochs=EPOCHS, steps_per_epoch = steps, callbacks=[early_stopping])

Epoch 1/5
128/128 [==============================] - 18s 86ms/step - loss: -0.9756
Epoch 2/5
128/128 [==============================] - 11s 83ms/step - loss: -1.0000
Epoch 3/5
128/128 [==============================] - 11s 85ms/step - loss: -1.0000
Epoch 4/5
128/128 [==============================] - 11s 83ms/step - loss: -1.0000
Epoch 5/5
128/128 [==============================] - 10s 81ms/step - loss: -1.0000


In [126]:
input_1 = (1, 8, 8, 8, 1)
input_1.__class__

tuple

In [3]:
import numpy as np

In [97]:
x = 10 * np.random.random(input_1)
print(x.shape)
print(x.__class__)

(1, 8, 8, 8, 1)
<class 'numpy.ndarray'>


In [67]:
y = input_1[1:]

In [93]:
a = (*block_shape, 1)
print(a)
print(a.__class__)

(8, 8, 8, 1)
<class 'tuple'>


In [31]:
print((np.random.random((10, 256)).shape))

(10, 256)


In [108]:
 y1 = encoder(x[1:])
 y1.shape
 #n_classes=n_classes)

TensorShape([0, 2048])

In [111]:
sh = y1.get_shape().as_list()
print(sh[1])

2048


In [112]:
assert sh[1] == projection_dim

In [ ]:
!pip install pytest

In [22]:
import pytest

In [113]:
input_shape=(1, 32, 32, 32, 1)
x = 10 * np.random.random(input_shape)

n_classes=1
weight_decay = 0.0005
projection_dim = 2048
latent_dim = 512
    

In [114]:
encoder, predictor = brainsiam(
      n_classes,
      input_shape=input_shape[1:],
      weight_decay=weight_decay,
      projection_dim=projection_dim,
      latent_dim=latent_dim,
    )

projection dimension is:  2048
latent dimension:  512
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 32, 32, 32, 1)]   0         
_________________________________________________________________
highresnet (Functional)      (None, 32, 32, 32, 1)     874785    
_________________________________________________________________
backbone_pool (GlobalAverage (None, 1)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 2048)              2048      
_________________________________________________________________
batch_normalization_81 (Batc (None, 2048)              8192      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 2048)              0         
_________________________________________________________________
dense

In [121]:
predictor.summary()

Model: "predictor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               1048576   
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_83 (Batc (None, 512)               2048      
_________________________________________________________________
dense_15 (Dense)             (None, 2048)              1050624   
Total params: 2,101,248
Trainable params: 2,100,224
Non-trainable params: 1,024
_________________________________________________________________


In [115]:
encoder_output = encoder(x[1:])
enc_output_shape = encoder_output.get_shape().as_list()
enc_output_shape

[0, 2048]

In [132]:
projection_dim = 2048

In [120]:
assert enc_output_shape[1] == projection_dim, 'encoder output shape not the same as projection dim'

In [123]:
predictor_out = predictor(encoder_output)

In [125]:
pred_output_shape = predictor_out.get_shape().as_list()
pred_output_shape

[0, 2048]

In [133]:
assert pred_output_shape[1] == projection_dim, 'predictor output shape not the same as projection dim'